In [ ]:
#!pip install libauc

In [ ]:
from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import DenseNet121, DenseNet169
from libauc.datasets import CheXpert

import torch
import os
import pandas as pd
from glob import glob
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score
import random
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torch import nn
from datasets import CheXpertDataset
from model import MultiLabelClassification
from torch.autograd import Variable
from sklearn.model_selection import KFold

In [ ]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
SEED = 123
set_all_seeds(SEED)

In [ ]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'data')
CheXpert_train_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_hidden_features_all.npy'))
CheXpert_train_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_labels_all.npy'))
CheXpert_valid_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_hidden_features_all.npy'))
CheXpert_valid_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_labels_all.npy'))
extra_valid_age_sex_df = pd.read_csv(os.path.join(DATA_DIR,'extra_valid_age_sex.csv'))
extra_valid_hidden_features = np.load(os.path.join(DATA_DIR,'extra_valid_hidden_features.npy'))
extra_valid_labels = np.load(os.path.join(DATA_DIR,'extra_valid_labels.npy'))
extra_valid_images = glob(os.path.join(DATA_DIR, 'extraValid', '*'))

In [ ]:
def smooth_labels(method, labels):
    if method == "ones":
        labels[labels == -1] = 1
    elif method == "ones-lsr":
        for i in range(labels.shape[0]):
            for j in range(labels.shape[1]):
                if labels[i,j] == -1:
                    labels[i,j] = random.uniform(.55, .85)
    elif method == "zeros":
        labels[labels == -1] = 0
        labels[labels == -1] = 0
    return labels

In [ ]:
def train(train_loader, val_loader, num_features, num_classes):
    
    imratio = 0.3424
    lr = 0.05 # using smaller learning rate is better
    gamma = 500
    weight_decay = 1e-5
    margin = 1.0

    # model
    model = MultiLabelClassification(num_feature=num_features, num_class=num_classes)
    model = model.cuda()
    
    # define loss & optimizer
    Loss = AUCMLoss(imratio=imratio)
    optimizer = PESG(model, 
                     a=Loss.a, 
                     b=Loss.b, 
                     alpha=Loss.alpha, 
                     imratio=imratio, 
                     lr=lr, 
                     gamma=gamma, 
                     margin=margin, 
                     weight_decay=weight_decay)

    best_val_auc = 0
    for epoch in range(2):
        if epoch > 0:
             optimizer.update_regularizer(decay_factor=10)
        for idx, data in enumerate(train_loader):
            train_data, train_labels = data
            train_data, train_labels = train_data.cuda(), train_labels.cuda()
            y_pred = model(train_data)
            loss = Loss(y_pred, train_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # validation
            if idx % 400 == 0:
                model.eval()
                with torch.no_grad():    
                    test_pred = []
                    test_true = [] 
                    for jdx, data in enumerate(val_loader):
                        test_data, test_label = data
                        test_data = test_data.cuda()
                        y_pred = model(test_data)
                        test_pred.append(y_pred.cpu().detach().numpy())
                        test_true.append(test_label.numpy())

                    test_true = np.concatenate(test_true)
                    test_pred = np.concatenate(test_pred)
                    val_auc =  roc_auc_score(test_true, test_pred) 
                    model.train()

                    if best_val_auc < val_auc:
                        best_val_auc = val_auc

    print ('Best Val_AUC is %.4f'%best_val_auc)
    return best_val_auc


In [ ]:
CheXpert_X = np.concatenate((CheXpert_train_hidden_features_all, CheXpert_valid_hidden_features_all), axis=0)
CheXpert_y = np.concatenate((CheXpert_train_labels_all, CheXpert_valid_labels_all), axis=0)

In [ ]:
#Performing some sort of label smoothing
CheXpert_y = smooth_labels("ones", CheXpert_y)
extra_valid_labels = smooth_labels("ones", extra_valid_labels)

In [ ]:
kf1 = KFold(n_splits=5, shuffle=True, random_state=SEED)
extra_train_indices = []
extra_val_indices = []
for train_index, val_index in kf1.split(extra_valid_hidden_features):
    extra_train_indices.append(train_index)
    extra_val_indices.append(val_index)

In [ ]:
kf2 = KFold(n_splits=5, shuffle=True, random_state=SEED)
idx = 0
aucs = []
for train_index, val_index in kf2.split(CheXpert_X):
    
    # Get CheXpert train and test datasets
    X_train_CheXpert, X_val_CheXpert = CheXpert_X[train_index], CheXpert_X[val_index]
    y_train_CheXpert, y_val_CheXpert = CheXpert_y[train_index], CheXpert_y[val_index]

    # Get hidden train and test datasets
    X_train_extra, X_val_extra = extra_valid_hidden_features[extra_train_indices[idx]], extra_valid_hidden_features[extra_val_indices[idx]]
    y_train_extra, y_val_extra = extra_valid_labels[extra_train_indices[idx]], extra_valid_labels[extra_val_indices[idx]]
    
    # Combine and shuffle
    '''X_train = np.concatenate((X_train_CheXpert, X_train_extra), axis=0)
    np.random.shuffle(X_train)
    X_val = np.concatenate((X_val_CheXpert, X_val_extra), axis=0)
    np.random.shuffle(X_val)
    y_train = np.concatenate((y_train_CheXpert, y_train_extra), axis=0)
    np.random.shuffle(y_train)
    y_val = np.concatenate((y_val_CheXpert, y_val_extra), axis=0)
    np.random.shuffle(y_val)'''
    
    # Load into dataloaders
    train_set = CheXpertDataset(X_train_CheXpert, y=y_train_CheXpert, scale_X=True)
    val_set = CheXpertDataset(X_val_CheXpert, y=y_val_CheXpert, scale_X=True)
    train_loader = DataLoader(train_set,
                             batch_size=32,
                             shuffle=True)
    val_loader = DataLoader(val_set,
                            batch_size=32,
                            shuffle=False)
    
    num_features = X_train_CheXpert.shape[1]
    num_classes = y_train_CheXpert.shape[1]
    
    auc = train(train_loader, val_loader, num_features, num_classes)
    aucs.append(auc)
    # Training...
    idx += 1
print(f'Average AUC {np.mean(aucs)}')

In [ ]:
trainset = CheXpertDataset(CheXpert_train_hidden_features_all, y=CheXpert_train_labels_all, scale_X=True)
testset = CheXpertDataset(CheXpert_valid_hidden_features_all, y=CheXpert_valid_labels_all, scale_X=True)
trainloader = DataLoader(trainset,
                         batch_size=32,
                         shuffle=True,
                         num_workers=2)
testloader = DataLoader(testset,
                        batch_size=32,
                        shuffle=False,
                        num_workers=2)

In [ ]:
# paramaters
SEED = 123
BATCH_SIZE = 32
imratio = 0.3424
lr = 0.05 # using smaller learning rate is better
gamma = 500
weight_decay = 1e-5
margin = 1.0

# model
model = MultiLabelClassification(num_feature=1024, num_class=5)
model = model.cuda()

# define loss & optimizer
Loss = AUCMLoss(imratio=imratio)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 imratio=imratio, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay)

best_val_auc = 0
for epoch in range(2):
    if epoch > 0:
         optimizer.update_regularizer(decay_factor=10)
    for idx, data in enumerate(trainloader):
        train_data, train_labels = data
        train_data, train_labels = train_data.cuda(), train_labels.cuda()
        y_pred = model(train_data)
        loss = Loss(y_pred, train_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # validation
        if idx % 400 == 0:
            model.eval()
            with torch.no_grad():    
                test_pred = []
                test_true = [] 
                for jdx, data in enumerate(testloader):
                    test_data, test_label = data
                    test_data = test_data.cuda()
                    y_pred = model(test_data)
                    test_pred.append(y_pred.cpu().detach().numpy())
                    test_true.append(test_label.numpy())

                test_true = np.concatenate(test_true)
                test_pred = np.concatenate(test_pred)
                val_auc =  roc_auc_score(test_true, test_pred) 
                model.train()

                if best_val_auc < val_auc:
                    best_val_auc = val_auc

            print('Epoch=%s, BatchID=%s, Val_AUC=%.4f, lr=%.4f'%(epoch, idx, val_auc,  optimizer.lr))

print ('Best Val_AUC is %.4f'%best_val_auc)

In [ ]:
extratestset = ChexpertDataset(extra_valid_hidden_features, y=extra_valid_labels, scale_X=True)
extratestloader = DataLoader(extratestset,
                         batch_size=32,
                         shuffle=True,
                         num_workers=2)

In [ ]:
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for jdx, data in enumerate(extratestloader):
        test_data, test_label = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_label.numpy())
    
    
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    auc =  roc_auc_score(test_true, test_pred) 

    print(auc)

print ('Test AUC is %.4f'%auc)

In [ ]:
# Training on actual images

In [ ]:
#U-Ones-LSR
'''
extra_valid_age_sex_df
extra_valid_hidden_features
extra_valid_labels
extra_valid_images
'''

In [ ]:
extra_valid_age_sex_df['sex'].unique()

In [ ]:
extra_valid_age_sex_df.isnull().values.any()

In [ ]:
#Convert to numpy array and then switch M to 1 and F to 0
extra_valid_age_sex_df = extra_valid_age_sex_df.replace(['F', 'M', 'O'], [0, 1, 2])
extra_valid_age_sex_df = extra_valid_age_sex_df.apply(pd.to_numeric)

In [ ]:
extra_valid_age_sex_np = extra_valid_age_sex_df.to_numpy()

In [ ]:
# Now with the numpy array append to to hidden feature vectors
print(extra_valid_age_sex_np.shape, extra_valid_hidden_features.shape)
hidden_with_age_sex = np.concatenate((extra_valid_hidden_features, extra_valid_age_sex_np), axis=1)

In [ ]:
# Split into train, and validation split
